In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import npc_sessions

In [ ]:
from npc_sessions import utils
import numpy as np
import rich

In [ ]:
session = npc_sessions.Session(
    R'\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2023-08-03_09-39-59_670248'
    )

Some stuff to help print results

In [ ]:
bad_tag = '[bold red]'
good_tag = '[bold green]'
cautious_tag = '[bold yellow]'

def bad_string(base: str):
    return bad_tag + base + bad_tag

def good_string(base: str):
    return good_tag + base + good_tag

def cautious_string(base: str):
    return cautious_tag + base + cautious_tag

def determine_string_valence(value, good_criterion, bad_criterion):
    if good_criterion:
        return good_string

    elif bad_criterion:
        return bad_string

    else:
        return cautious_string

def add_valence_to_string(basestring: str, value: float | int, good_criterion: bool, bad_criterion: bool) -> str:

    valence_func = determine_string_valence(value, good_criterion, bad_criterion)

    return valence_func(basestring)


# Sync

In [ ]:
devices = utils.get_ephys_timing_on_pxi(session.ephys_recording_dirs)
for device in devices:
        (
            ephys_barcode_times,
            ephys_barcode_ids,
        ) = utils.extract_barcodes_from_times(
            on_times=device.ttl_sample_numbers[device.ttl_states > 0]
            / device.sampling_rate,
            off_times=device.ttl_sample_numbers[device.ttl_states < 0]
            / device.sampling_rate,
        )
        plt.plot(np.diff(ephys_barcode_times))

### barcodes

In [ ]:
full_exp_recording_dirs = [utils.get_single_oebin_path(directory).parent for directory in session.ephys_record_node_dirs]
display(tuple((device.name, device.sampling_rate, device.start_time) for device in npc_sessions.get_ephys_timing_on_sync(session.sync_path, full_exp_recording_dirs)))

get barcode intervals for each probe

In [ ]:
barcode_rising = session.sync_data.get_rising_edges(0, 'seconds')
barcode_falling = session.sync_data.get_falling_edges(0, 'seconds')
barcode_times, barcodes = utils.extract_barcodes_from_times(barcode_rising, barcode_falling)

devices_pxi = utils.get_ephys_timing_on_pxi(full_exp_recording_dirs)
devices_sync = tuple(utils.get_ephys_timing_on_sync(session.sync_path, session.ephys_recording_dirs))
device_barcode_dict = {}
for device in devices_pxi:
        if 'NI-DAQmx' in device.name or 'LFP' in device.name:
            continue
        
        device_sync = [d for d in devices_sync if d.name==device.name][0]

        (
            ephys_barcode_times,
            ephys_barcode_ids,
        ) = utils.extract_barcodes_from_times(
            on_times=device.ttl_sample_numbers[device.ttl_states > 0]
            / device.sampling_rate,
            off_times=device.ttl_sample_numbers[device.ttl_states < 0]
            / device.sampling_rate,
        )
        raw = ephys_barcode_times
        corrected = ephys_barcode_times*(30000/device_sync.sampling_rate)
        intervals = np.diff(corrected)
        max_deviation = np.max(np.abs(intervals - np.median(intervals)))

        device_barcode_dict[device.name] = {'barcode_times_raw': raw, 
                                            'barcode_times_corrected': corrected,
                                            'max_deviation_from_median_interval': max_deviation}

Plot barcode intervals for sync and for each probe after sample rate correction

In [ ]:
fig, ax = plt.subplots(1,3)
fig.set_size_inches([8, 4])
sync_intervals = np.diff(barcode_times)
sync_max_deviation_from_median_interval = np.max(np.abs(sync_intervals - np.median(sync_intervals)))
sync_max_deviation_string =  add_valence_to_string(f'Sync deviation: {sync_max_deviation_from_median_interval}',
                                                        sync_max_deviation_from_median_interval,
                                                        sync_max_deviation_from_median_interval<0.001,
                                                        sync_max_deviation_from_median_interval>0.001)
rich.print(sync_max_deviation_string)

ax[0].plot(sync_intervals)
legend = []
for device_name, device_data in device_barcode_dict.items():
    ax[1].plot(np.diff(device_data['barcode_times_raw']))
    ax[2].plot(np.diff(device_data['barcode_times_corrected']))
    legend.append(device_name.split('Probe')[1])
    max_deviation = device_data['max_deviation_from_median_interval']
    max_deviation_string =  add_valence_to_string(f'{device_name}: {max_deviation}',
                                                        max_deviation,
                                                        max_deviation<0.001,
                                                        max_deviation>0.001)
    
    rich.print(max_deviation_string)



ax[2].plot(sync_intervals, 'k')
ax[2].legend(legend + ['sync'])
ax[0].set_title('Sync Barcode Intervals')
ax[1].set_title('Probe Barcode Intervals')
ax[2].set_title('Probe Barcode Intervals Corrected')

plt.tight_layout()

## stim timing

Plot vsync times and diode flips for the beginnings and ends of each stimulus epoch

In [ ]:
rich.print('[bold] Fraction long frames [/bold]')
for stim_name, stim_times in stim_display_times.items():
    intervals = np.diff(stim_times)
    fraction_long = np.sum(intervals>0.02)/len(intervals)
    longest_interval = max(intervals)
    start_tag, end_tag = ('[bold green]', '[/bold green]') if fraction_long<0.01 and longest_interval<0.5 else ('[bold magenta]', '[/bold magenta]')
    rich.print(start_tag + stim_name.stem.split('_')[0] + ': ' + str(fraction_long) + ' \t\t longest interval:' + str(longest_interval) + end_tag)
    

plot diode flips

In [ ]:
fig, axes = session.sync_data.plot_diode_measured_sync_square_flips()
stim_display_times = npc_sessions.get_stim_frame_times(*session.stim_paths, sync=session.sync_data)
names = tuple(k for k, v in stim_display_times.items() if v is not None)
for idx, ax in enumerate(axes):
    ax.set_title(names[idx].stem.split('_')[0])
fig.set_size_inches(12, 6)

plot histogram of vsync intervals
TO DO: add histogram of diode flip intervals

In [ ]:
fig_hist, axes_hist = plt.subplots(1, len(axes))
fig_hist.set_size_inches(12, 6)

for ax, (stim_name, stim_times) in zip(axes_hist, stim_display_times.items()):
    ax.hist(np.diff(stim_times), bins=np.arange(0, 0.1, 0.001))
    ax.set_yscale('log')
    ax.axvline(1/60, c='k', ls='dotted')
    ax.set_title(stim_name.stem.split('_')[0])
    ax.set_xlabel('time (s)')
    ax.set_ylabel('frame interval count')
plt.tight_layout()

In [ ]:
fig, axes = session.sync_data.plot_stim_onsets()
names = tuple(k for k, v in stim_display_times.items() if v is not None)
for idx, ax in enumerate(axes):
    ax.set_title(names[idx].stem.split('_')[0])
fig.set_size_inches(10, 5 * len(axes))
fig.subplots_adjust(hspace=0.3)


Plot indicating where long frames happened relative to stim times during task

Generate report indicating how many long frames we got and whether there were weird things with the diode (vsync number vs diode flip number, how many blips we got on the diode line etc)

Plot sound latencies/durations across session

# MVR

In [ ]:
import IPython.display

for info in session.video_info_data.values():
    print(info['CameraLabel']), display(IPython.display.JSON(
        {
            k:v
            for k,v in info.items() 
            if k in ('FPS', 'Duration', 'FramesRecorded', 'FramesLostCount')
        } | {'Lost': f"{100 * info['FramesLostCount'] / info['FramesRecorded']:.5f}%"}
    )
)

Plot frames taken across session

In [ ]:
from typing import Iterable
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
import numpy as np
import datetime
import io
from typing import Literal, TypeVar

import upath

def camera_frame_grabs_simple(
    paths: Iterable[upath.UPath],
    num_frames_to_grab: int = 5,
) -> None:
    """Just plots evenly spaced frames, no concept of epochs.
    
    video frames across cameras aren't synced .
    """
    fig = plt.figure(figsize=[10, 3 * len(paths)], constrained_layout=True, facecolor='0.5')
    gs = gridspec.GridSpec(len(paths), num_frames_to_grab, figure=fig)
    gs.update(wspace=0.0, hspace=0.0)
    for idx, video_path in enumerate(paths):
        # get frames to plot
        v = cv2.VideoCapture(video_path.as_posix()) # TODO open with upath from cloud
        
        frame_delta = np.ceil(v.get(cv2.CAP_PROP_FRAME_COUNT) / num_frames_to_grab + 1)
        frames_of_interest = np.arange(v.get(cv2.CAP_PROP_FPS), v.get(cv2.CAP_PROP_FRAME_COUNT), frame_delta)

        for i, f in enumerate(frames_of_interest):
            v.set(cv2.CAP_PROP_POS_FRAMES, int(f))
            ret, frame = v.read()
            ax = fig.add_subplot(gs[idx, i])
            ax.imshow(frame)
            # ax.axis('off')
            ax.tick_params(
                top=False,
                bottom=False,
                left=False,
                right=False,
                labelleft=False,
                labelbottom=False,
            )
            ax.set_title(datetime.timedelta(seconds=f/v.get(cv2.CAP_PROP_FPS)), fontsize=10)
    return fig


fig = camera_frame_grabs_simple(session.video_paths)

Verify frame times can be found on sync: generate report describing frame rate/lost frames/discrepancy between frames expected from sync and frames in video

In [ ]:
from npc_sessions.utils.mvr import get_augmented_camera_info

In [ ]:
augmented_camera_info = get_augmented_camera_info(session.sync_data, *session.video_paths)

for camera, info in augmented_camera_info.items():
    rich.print(f'[bold]{camera} camera stats[bold]')
    
    frame_rate = info['FPS']
    frame_rate_string = add_valence_to_string(f'Frame Rate: {frame_rate} \t',
                                                frame_rate,
                                                abs(frame_rate-60)<0.01, 
                                                abs(frame_rate-60)>0.05)
    
    lost_frame_percentage = 100 * info['FramesLostCount'] / info['FramesRecorded']
    lost_frame_string = add_valence_to_string(f'Lost frame percentage: {np.round(lost_frame_percentage, 3)} \t',
                                                lost_frame_percentage,
                                                lost_frame_percentage<0.01, 
                                                lost_frame_percentage>0.05)

    frame_diff_from_expected = info['expected_minus_actual']
    frame_diff_string = add_valence_to_string(f'Frames expected minus actual: {frame_diff_from_expected}',
                                                frame_diff_from_expected,
                                                abs(frame_diff_from_expected)<1, 
                                                abs(frame_diff_from_expected)>10)



    rich.print(frame_rate_string + lost_frame_string + frame_diff_string)




Plot video frames that should have licks

In [ ]:
lick_times = session._licks.timestamps
video_frame_times = npc_sessions.get_video_frame_times(session.sync_data, *session.video_paths)



## Recording Quality

Drift raster for each probe

Quality metric histograms for each probe (isi violoations, amplitude cutoff, drift ptp)

all spike histograms for each probe

## behavior

Plot behavior across blocks (Plots in Ethan's notebook)

Plot lick latencies

Cumulative reward volume over session

plot running across session

## Basic Sync Report

Generate report showing 1) number of edges, 2) median edge interval 3) overall edge frequency for each data line in sync file